# COMP4030 - Data Science and Machine Learning - Coursework 2

In [1]:
from utils.gestures import *
from utils.operations import *
from utils.files import * 

import numpy as np
import pandas as pd

Firstly we ensure that the raw data provided by Phyphox is unzipped and ready for import.

In [2]:
process_all_zips()

No unprocessed files found for circle
No unprocessed files found for come
No unprocessed files found for go
No unprocessed files found for wave


Next we read the raw data from the CSV files and place the data inside a pandas DataFrame. We also use this opportunity to normalise our data using a MinMaxScaler, ensuring that all of our data lies between 0 and 1. This will ensure that no single field plays a more important role than it should.

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# scaler = StandardScaler()
scaler = MinMaxScaler()

df, test_data = get_data_from_files(scaler, 10, test_percentage=0.2) # 10 is the rows to trim from the beginning and end of the data
df

,time,accel_x,accel_y,accel_z,accel_abs,file_number,gesture
0,0.000000,0.481011,0.525357,0.150730,0.094869,3,circle
1,0.000816,0.507888,0.574575,0.210860,0.041192,3,circle
2,0.001632,0.550675,0.605845,0.307853,0.127547,3,circle
3,0.002449,0.565785,0.605547,0.297189,0.125086,3,circle
4,0.003265,0.548515,0.574028,0.218159,0.036035,3,circle
...,...,...,...,...,...,...,...
119503,0.997606,0.520334,0.913148,0.593166,0.001208,13,wave
119504,0.998205,0.519230,0.912340,0.590457,0.002987,13,wave
119505,0.998803,0.518487,0.908197,0.587279,0.005490,13,wave
119506,0.999402,0.518832,0.907147,0.585266,0.006249,13,wave


Next we visualise our raw data for exploratory analysis

In [4]:
test_data

,time,accel_x,accel_y,accel_z,accel_abs,file_number,gesture
0,0.000000,0.559598,0.513468,0.226458,0.092301,0,circle
1,0.000728,0.568016,0.522626,0.197835,0.093494,0,circle
2,0.001457,0.517814,0.566211,0.222524,0.009878,0,circle
3,0.002185,0.484999,0.581607,0.242733,0.035518,0,circle
4,0.002913,0.510461,0.583776,0.212801,0.023593,0,circle
...,...,...,...,...,...,...,...
17740,0.996686,0.449168,0.752667,0.447974,0.007651,2,wave
17741,0.997515,0.450282,0.751411,0.440516,0.008550,2,wave
17742,0.998343,0.451031,0.748220,0.442095,0.006494,2,wave
17743,0.999172,0.450893,0.747173,0.439946,0.007358,2,wave


In [5]:
SHOW_RAW_DATA = False

In [6]:
import math
import matplotlib.pyplot as plt

if SHOW_RAW_DATA:
      for gesture in get_gestures():
            
            group = df[df['gesture'] == gesture].groupby('file_number')
            
            if len(group) == 0:
                  continue

            cols = 2
            rows = math.ceil(len(group) / cols)
            plt.figure(figsize=(cols * 15, rows * 3))

            for index, df_group in enumerate(group):
                  plt.subplot(rows, cols, index + 1)
                  for col in get_columns()[1:]:
                        plt.plot(df_group[1].index, df_group[1][col], label=col)
                        plt.xticks(range(df_group[1].index[0], df_group[1].index[-1], 50))
                        plt.legend()
            plt.suptitle(gesture)
            plt.show()

We then try to segment this data, using the dips in acceleration that we had purposefully included in our data captures.

Firstly we visualise just one example to inspect the rolling standard deviation and attempt to find the optimal parameters for slicing our data.

In [7]:
rolling_window = 10
group_threshold = 10
deviation_threshold = 1.7

In [21]:
plt.figure(figsize=(30, 10))

for i, gesture in enumerate(get_gestures()):

      group = df[(df['gesture'] == gesture) & (df['file_number'] == 1)]
      if len(group) == 0:
            continue

      rolling_std = group['accel_abs'].rolling(window=rolling_window).std()
      threshold = rolling_std.mean() - rolling_std.std() / deviation_threshold

      plt.subplot(2, 2, i + 1)

      for col in get_columns()[1:]:
            plt.plot(group[col].rolling(window=rolling_window).std(), label=f'Rolling Std {col}')

      plt.axhline(y=rolling_std.mean(), color='r', linestyle='-', label='Mean')
      plt.axhline(y=threshold, color='g', linestyle='--', label='Potential Threshold')    
      plt.title(f'Rolling Standard Deviation {gesture}')
      plt.ylabel('Standard Deviation')
      plt.legend()

plt.show()

<Figure size 3000x1000 with 0 Axes>

Then we visualise our slices for all of the data.

In [9]:
plt.figure(figsize=(30, 12))
plot = 1
for gesture in get_gestures():
    feature = df[df['gesture'] == gesture]
    for i, sub_group in feature.groupby('file_number'):

        below_threshold = indices_below_threshold(sub_group, rolling_window, group_threshold, deviation_threshold)
        breakpoints = merge_and_average_integers(below_threshold, 25)

        if i > 1:
            break

        plt.subplot(4, 2, plot)
        plot += 1

        for col in get_columns()[1:]:
            plt.plot(sub_group.index, sub_group[col], label=col)

        for index in breakpoints:
            plt.axvline(x=index, color='r', linestyle='--')

        plt.ylabel('Acceleration')
        plt.title(f'{gesture} No.{i}')
        plt.legend()
        plt.subplots_adjust(hspace=0.45)
    
plt.suptitle("Segments of gestures with breakpoints")
plt.show()

<Figure size 3000x1200 with 0 Axes>

We can safely remove the data where the absolute acceleration never gets above a certain threshold.

In [10]:
label_gestures(df, 'accel_abs',  0.3, rolling_window = 10, group_threshold = 5, deviation_threshold = deviation_threshold)

In [11]:
df

,time,accel_x,accel_y,accel_z,accel_abs,file_number,gesture,gesture_number
0,0.029387,0.528114,0.558846,0.184554,0.012171,3,circle,1.0
1,0.030204,0.516467,0.558169,0.184049,0.023389,3,circle,1.0
2,0.031021,0.522136,0.563239,0.178458,0.022435,3,circle,1.0
3,0.031837,0.531654,0.573794,0.189354,0.022983,3,circle,1.0
4,0.032653,0.532224,0.574294,0.195692,0.022442,3,circle,1.0
...,...,...,...,...,...,...,...,...
98466,0.970676,0.533817,0.923186,0.601583,0.022988,13,wave,422.0
98467,0.971275,0.534736,0.921991,0.603199,0.024799,13,wave,422.0
98468,0.971873,0.534174,0.921728,0.603683,0.024011,13,wave,422.0
98469,0.972472,0.533964,0.918852,0.602903,0.023259,13,wave,422.0


In [12]:
plt.figure(figsize=(30, 12))
plot = 1
for gesture in get_gestures():
    feature = df[df['gesture'] == gesture]
    for i, sub_group in feature.groupby('file_number'):

        if i > 1:
            break

        plt.subplot(4, 2, plot)
        plot += 1

        for gesture_number in sub_group['gesture_number'].unique():
            gesture_group = sub_group[sub_group['gesture_number'] == gesture_number]
            # plt.plot(gesture_group.index, gesture_group['accel_abs'], label=number_to_color(gesture))
        
            for col in get_columns()[1:]:
                plt.plot(gesture_group.index, gesture_group[col], label=number_to_color(gesture_number))

        plt.ylabel('Acceleration')
        plt.title(f'{gesture} No.{i}')
        plt.subplots_adjust(hspace=0.45)
    
plt.suptitle("Segments of gestures with breakpoints")
plt.show()

<Figure size 3000x1200 with 0 Axes>

In [23]:
for label, group in df.groupby(['gesture', 'gesture_number']):
      if(group.shape[0] < 50):
            data = df[(df['gesture'] == label[0]) & (df['gesture_number'] == label[1])]
            print(label, data['file_number'].unique(), data['gesture_number'].unique())
            

('circle', 38.0) [4] [38.]
('circle', 92.0) [5] [92.]
('circle', 353.0) [11] [353.]


Safest thing to do is to drop the first and last gesture from the data

In [14]:
from sklearn.preprocessing import LabelEncoder

def encode_labels(labels):
    encoder = LabelEncoder()
    return encoder.fit_transform(labels)

def extract_features(group, sub_blocks = 20):
    (gesture, gesture_number), data = group
    features = []

    block_size = math.ceil(len(data) / sub_blocks)

    ## Summarise all rows into a 1D array which represents n sub-blocks of the data
    for i in range(sub_blocks):
        start_index = i * block_size
        end_index = min((i + 1) * block_size, len(data))
        sub_block = data.iloc[start_index:end_index]
        
        for axis in get_columns()[1:]:
            features.append(sub_block[axis].mean())
            features.append(sub_block[axis].std())

    return features, gesture

def get_features_and_labels():
    features = []
    original_labels = []

    extracted_features = [extract_features(group) for group in df.groupby(['gesture', 'gesture_number'])]
    for feature, label  in extracted_features:
        features.append(feature)
        original_labels.append(label)

    return features, original_labels

features, original_labels = get_features_and_labels()
labels = encode_labels(original_labels)

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [16]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)

In [17]:
model.fit(features, labels)

RandomForestClassifier(random_state=42)

In [18]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))


Accuracy: 0.9294117647058824
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        33
           1       0.91      0.83      0.87        12
           2       0.84      0.94      0.89        17
           3       1.00      0.87      0.93        23

    accuracy                           0.93        85
   macro avg       0.92      0.91      0.91        85
weighted avg       0.93      0.93      0.93        85



In [19]:
model.predict(X_test[0:1])

array([1], dtype=int64)

In [20]:
model.predict(test_data)

c:\Users\Ashley\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: 'circle'